In [83]:
from pprint import pprint

def main():
    N = 8
    K = 4
    grid = np.array([[[0,0,0,0] for _ in range(N)] for _ in range(N)])

    dataset = {}

    # horizontal

    for i in range(N):
        for j in range(N):
            grid[i][j] = [0,1,0,0]
        if 1 not in dataset:
            dataset[1] = [grid]
        else:
            dataset[1].append(grid)
        
        grid = np.array([[[0,0,0,0] for _ in range(N)] for _ in range(N)])

    # vertical

    for i in range(N):
        for j in range(N):
            grid[j][i] = [1,0,0,0]
            if 2 not in dataset:
                dataset[2] = [grid]
        else:
            dataset[2].append(grid)
        grid = np.array([[[0,0,0,0] for _ in range(N)] for _ in range(N)])

    # ldiagonal - da
    
    row = 0
    col = 0

    for i in range(N):
        col = i
        row = 0

        while (col >= 0 and row < N):
            grid[row][col] = [0,0,1,0]
            row += 1
            col -= 1
        if 3 not in dataset:
            dataset[3] = [grid]
        else:
            dataset[3].append(grid)
        grid = np.array([[[0,0,0,0] for _ in range(N)] for _ in range(N)])

    
    for i in range(1, N):
        row = i
        col = N - 1

        while (col >= 0 and row < N):
            grid[row][col] = [0,0,1,0]
            row += 1
            col -= 1
        if 3 not in dataset:
            dataset[3] = [grid]
        else:
            dataset[3].append(grid)
        
        grid = np.array([[[0,0,0,0] for _ in range(N)] for _ in range(N)])

    # rdiagonal - db

    for i in range(N):
        col = N - 1 - i
        row = 0

        while (col < N and row < N):
            grid[row][col] = [0,0,0,1]
            row += 1
            col += 1
        if 4 not in dataset:
            dataset[4] = [grid]
        else:
            dataset[4].append(grid)
        
        grid = np.array([[[0,0,0,0] for _ in range(N)] for _ in range(N)])

        

    for i in range(1, N):
        row = i
        col = 0

        while (col >= 0 and row < N):
            grid[row][col] = [0,0,0,1]
            row += 1
            col += 1
        if 4 not in dataset:
            dataset[4] = [grid]
        else:
            dataset[4].append(grid)
        
        grid = np.array([[[0,0,0,0] for _ in range(N)] for _ in range(N)])
    
    return dataset

        
            
if __name__ == '__main__':
    main()

In [155]:
from random import random
import numpy as np

num_complex = 4
delta = 0.01
alpha = 0.1

In [156]:
def make_grid(n):
    return [[make_simple_unit() for i in range(n)] for j in range(n)]

def make_simple_unit():
    return {'v': initialize(),'h':initialize(),'da':initialize(),'db':initialize()}

#firing is x, one weight for each connection to each complex cell
def initialize():
    return {'firing':0, 'weights':[random() for i in range(num_complex)]}

#firing is y, trace is ybar
def make_complex_unit():
    return {'firing':0,'trace':random()}

def make_complex_layer():
    return [make_complex_unit() for i in range(num_complex)]

def find_weight_change(trace,firing,weight):
    return alpha*trace*(firing - weight)

def find_new_trace(firing, trace):
    return (1-delta)*trace + delta*firing    

#determines which complex cell should fire
def which_firing(grid, complex_layer):
    sums = {i:0 for i in range(num_complex)}
    for i,row in enumerate(grid):
        for j,unit in enumerate(row):
            for orientation in unit:
                firing = unit[orientation]['firing']
                if (firing == 1):
                    for complex_cell_num in sums:
                        sums[complex_cell_num] += firing * unit[orientation]['weights'][complex_cell_num]
                weights = unit[orientation]['weights']
                for complex_cell_num, weight in enumerate(weights):
                    grid[i][j][orientation]['weights'][complex_cell_num] += find_weight_change(complex_layer[complex_cell_num]['trace'],firing,weight)
    inverse = [(value, key) for key, value in sums.iteritems()]
    return max(inverse)[1]

#find instances of firing feature detectors in a frame (firing feature detectors -> "ffds")
def read_frame(frame):
    ffds = []
    for (row_num,col_num,feature_detector),is_firing in np.ndenumerate(frame):
        if (is_firing == 1):
            ffds.append((row_num,col_num,feature_detector))
    return ffds

def run_model(frames):
    grid = make_grid(len(frames[0]))
    complex_layer = make_complex_layer()
    print complex_layer
    fd2orient = {0:'v', 1:'h', 2:'da', 3:'db'}
    complex_num_firing = 0
    for i in range(10):
        for frame in frames:
            complex_layer[complex_num_firing]['firing'] = 0
            ffds = read_frame(frame)
            for index in ffds:
                row = index[0]
                col = index[1]
                orientation = fd2orient[index[2]]
                grid[row][col][orientation]['firing'] = 1
            complex_num_firing = which_firing(grid, complex_layer)
            complex_layer[complex_num_firing]['firing'] = 1
            for complex_unit in complex_layer:
                complex_unit['trace'] = find_new_trace(complex_unit['firing'],complex_unit['trace'])  
        print complex_layer

In [148]:
data = main()
hframes = data[1]
vframes = data[2]
daframes = data[3]
dbframes = data[4]

In [159]:
run_model(vframes)

[{'firing': 0, 'trace': 0.09013052334956284}, {'firing': 0, 'trace': 0.09254694190516977}, {'firing': 0, 'trace': 0.5648656783125835}, {'firing': 0, 'trace': 0.5056099742644483}]
[{'firing': 0, 'trace': 0.08233578760455267}, {'firing': 1, 'trace': 0.1430630782312391}, {'firing': 0, 'trace': 0.5254293411441009}, {'firing': 0, 'trace': 0.4804315324136823}]
[{'firing': 0, 'trace': 0.07521516206190862}, {'firing': 0, 'trace': 0.13991803638661754}, {'firing': 0, 'trace': 0.4897897654691018}, {'firing': 1, 'trace': 0.5063367966669744}]
[{'firing': 0, 'trace': 0.06871034781583074}, {'firing': 0, 'trace': 0.12781753947321872}, {'firing': 0, 'trace': 0.4474313983969919}, {'firing': 1, 'trace': 0.5490301493072576}]
[{'firing': 0, 'trace': 0.06276808781036129}, {'firing': 0, 'trace': 0.11676352683970638}, {'firing': 0, 'trace': 0.4087362995013764}, {'firing': 1, 'trace': 0.5880312632970572}]
[{'firing': 0, 'trace': 0.057339730806332714}, {'firing': 0, 'trace': 0.10666549564509079}, {'firing': 0, 